In [436]:
import sklearn
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [437]:
df = pd.read_csv('titanic/train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### easy  
##### для данных с Титаника обучить модели логистической регрессии и решающего дерева на 80% обучающей выборки и проверить результат на 20%
##### у какой модели результат лучше?

отбор нужных данных, замена текстовых данных на числовые

In [438]:
df.loc[df['Sex']=='female', 'Sex'] = 1
df.loc[df['Sex']=='male', 'Sex'] = 0
df_ml = df[['Survived','Pclass','Sex','Age']].fillna(0)
df_ml

,Survived,Pclass,Sex,Age
0,0,3,0,22.0
1,1,1,1,38.0
2,1,3,1,26.0
3,1,1,1,35.0
4,0,3,0,35.0
...,...,...,...,...
886,0,2,0,27.0
887,1,1,1,19.0
888,0,3,1,0.0
889,1,1,0,26.0


определение x и y

In [439]:
x = df_ml[['Pclass','Sex','Age']]
x.head()

,Pclass,Sex,Age
0,3,0,22.0
1,1,1,38.0
2,3,1,26.0
3,1,1,35.0
4,3,0,35.0


In [440]:
y = df_ml[['Survived']]
y.head()

,Survived
0,0
1,1
2,1
3,1
4,0


разбивка на train и test (80% - 20%)

In [441]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8)

**обучение модели логистической регрессии**

In [442]:
model_1 = LogisticRegression(max_iter=1000)
model_1.fit(x_train, y_train)
y_pred = model_1.predict(x_test)
a1 = accuracy_score(y_pred, y_test)
a1

C:\Users\79216\anaconda3\envs\py38\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.7653631284916201

**обучение модели решающего дерева**

In [443]:
from sklearn.tree import DecisionTreeClassifier

In [444]:
model_2 = DecisionTreeClassifier()
model_2.fit(x_train, y_train)
y_pred = model_2.predict(x_test)
a2 = accuracy_score(y_pred, y_test)
a2

0.8156424581005587

**сравнение результатов**

In [445]:
print('модель логистической регрессии - ',round(a1*100,2),'%')
print('модель решающего дерева - ',round(a2*100,2),'%')

модель логистической регрессии -  76.54 %
модель решающего дерева -  81.56 %


при нормализации данных точность значительно падает:

In [446]:
from sklearn import preprocessing

In [447]:
x_n=preprocessing.normalize(x_df)

In [448]:
x_train_n, x_test_n, y_train_n, y_test_n = train_test_split(x_n, y, train_size=0.8)

In [449]:
model_1n = LogisticRegression(max_iter=1000)
model_1n.fit(x_train_n, y_train_n)
y_pred_n = model_1n.predict(x_test_n)
accuracy_score(y_pred_n, y_test_n)

C:\Users\79216\anaconda3\envs\py38\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.664804469273743

In [450]:
model_2n = DecisionTreeClassifier()
model_2n.fit(x_train, y_train)
y_pred = model_2n.predict(x_test)
accuracy_score(y_pred, y_test)

0.8156424581005587

### normal  
##### используя кросс-валидацию, применить к данным Титаника модель RandomForestClassifier; сравнить с первыми двумя моделями

**обучение модели RandomForestClassifier**

In [451]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [452]:
model_3 = DecisionTreeClassifier()
model_3.fit(x_train, y_train)
y_pred = model_3.predict(x_test)
a3 = accuracy_score(y_pred, y_test)

In [453]:
cr_val = cross_val_score(model_3, x_train, y_train)
a4 = accuracy_score(model_3.predict(x_test), y_test)
print('кросс-валидация на train выборке:',cr_val)
print('cреднее по кросс-валидации:', cr_val.mean())
print('качество на тестовой выборке:', accuracy_score(model_3.predict(x_test), y_test))

кросс-валидация на train выборке: [0.7972028  0.7972028  0.78169014 0.78169014 0.76760563]
cреднее по кросс-валидации: 0.7850783019797104
качество на тестовой выборке: 0.8156424581005587


**сравнение результатов**

In [454]:
print('модель логистической регрессии - ',round(a1*100,2),'%')
print('модель решающего дерева - ',round(a2*100,2),'%')
print('модель RandomForestClassifier - ',round(a3*100,2),'%')
print('модель RandomForestClassifier с кросс-валидацией - ',round(a4*100,2),'%')

модель логистической регрессии -  76.54 %
модель решающего дерева -  81.56 %
модель RandomForestClassifier -  81.56 %
модель RandomForestClassifier с кросс-валидацией -  81.56 %


### hard (недоделано)  
##### реальзовать линейную регрессию  

In [466]:
df_wh = pd.read_csv('weight-height.csv')
df_wh.head()

,Gender,Height,Weight
0,Male,73.847017,241.893563
1,Male,68.781904,162.310473
2,Male,74.110105,212.740856
3,Male,71.730978,220.042470
4,Male,69.881796,206.349801


**определение входных данных /х/**

In [467]:
x_wh = df_wh[['Weight']]
x_wh

,Weight
0,241.893563
1,162.310473
2,212.740856
3,220.042470
4,206.349801
...,...
9995,136.777454
9996,170.867906
9997,128.475319
9998,163.852461


In [468]:
y_wh = df_wh[['Height']]
y_wh

,Height
0,73.847017
1,68.781904
2,74.110105
3,71.730978
4,69.881796
...,...
9995,66.172652
9996,67.067155
9997,63.867992
9998,69.034243


**расчёт весов, определение функции**

In [469]:
from sklearn.linear_model import LinearRegression

In [470]:
model_wh = LinearRegression()
model_wh.fit(x_wh,y_wh)
w = float(model_wh.coef_)
b = float(model_wh.intercept_)
print('веса: w = ', w, 'b = ',b)
print('функция: y = ',w,'*x + ',b)

веса: w =  0.11081279484017847 b =  48.47790261417956
функция: y =  0.11081279484017847 *x +  48.47790261417956


In [471]:
163.852461*w+b

66.63485175903091

**среднеквадратичное отклонение (ошибка)**

In [472]:
from sklearn.metrics import mean_squared_error

In [473]:
y_pred_wh = model_wh.predict(x_wh)
mse = mean_squared_error(y_wh, y_pred_wh)
print('среднеквадратичное отклонение (mse) = ',mse)

среднеквадратичное отклонение (mse) =  2.1437101968829455


**минимизация функции**  
вот здесь что-то пошло не так, всё время минимизация не успешна

In [474]:
from scipy.optimize import minimize

In [475]:
x = float(df_wh[['Weight']].mean())
def J(w,b):
    return x*w+b+mse

In [476]:
x_start = [0.1,0.1]
minimize(lambda x: J(x[0], x[1]), (x_start,))

      fun: -38080004782079.5
 hess_inv: array([[-7.04937234e+09, -4.36654123e+07],
       [-4.36654123e+07, -2.70472475e+05]])
      jac: array([161.44035623,   1.0001187 ])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 390
      nit: 2
     njev: 126
   status: 2
  success: False
        x: array([-2.35867564e+11, -1.46101723e+09])